<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/explorations1/explore_nbs/tabularTrainingTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
VERSION = "20200707" #"nightly"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
!python pytorch-xla-env-setup.py --version $VERSION > /dev/null
#import torch_xla.core.xla_model as xm

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  52193      0 --:--:-- --:--:-- --:--:-- 52193
Copying gs://tpu-pytorch/wheels/torch-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
- [1 files][107.5 MiB/107.5 MiB]                                                
Operation completed over 1 objects/107.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
\ [1 files][123.8 MiB/123.8 MiB]                                                
Operation completed over 1 objects/123.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.2 MiB/  2.2 MiB]                                                
Operation completed over 1 objects/2.2 MiB.                                      
ERROR:

In [2]:
!pip install https://github.com/butchland/fastai_xla_extensions/archive/master.zip > /dev/null
import fastai_xla_extensions.core

In [3]:
#!pip install fastai2 > /dev/null

In [4]:
from fastai2.tabular.all import *

In [5]:
dede = default_device()
dede

device(type='xla', index=1)

In [6]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [7]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    device = dede,
    procs = [Categorify, FillMissing, Normalize])
dls.device

device(type='xla', index=1)

In [8]:
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(15)

epoch,train_loss,valid_loss,accuracy,time
0,0.367259,0.350020,0.835227,00:23
1,0.370577,0.345900,0.835381,00:20
2,0.361738,0.344600,0.841063,00:19
3,0.357572,0.340278,0.842752,00:18
4,0.359882,0.343849,0.840295,00:19
5,0.346141,0.336789,0.846591,00:18
6,0.352187,0.339715,0.841984,00:18
7,0.347866,0.339989,0.844595,00:19
8,0.354371,0.337098,0.845823,00:18
9,0.341744,0.335808,0.846898,00:18


/usr/local/lib/python3.6/dist-packages/fastai2/callback/schedule.py:68: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx = (pos >= pcts).nonzero().max()


In [9]:
learn.predict(df.iloc[0])


(   workclass  education  marital-status  ...    fnlwgt  education-num  salary
 0        5.0        8.0             3.0  ... -0.838534       0.751553     1.0
 
 [1 rows x 11 columns], tensor(1), tensor([0.4120, 0.5880]))

In [10]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)
###
learn.get_preds(dl=dl)

(tensor([[0.4120, 0.5880],
         [0.2422, 0.7578],
         [0.9796, 0.0204],
         ...,
         [0.4187, 0.5813],
         [0.6817, 0.3183],
         [0.6503, 0.3497]]), None)